# Kubernetes

# Kubernetes Version (Current)

In [1]:
#!pip install ipynb
from ipynb.fs.full.facturxWidget import *

from ipynb.fs.full.kubernetesTool import *

from ipynb.fs.full.invoiceSmartcontract import *

## addPeer

### KubernetesPeer

In [12]:
import kubernetesTool
import subprocess

class KubernetesPeer: 
    def __init__(self, name='',prefix='peer', hostname='kubernetes.research.dev.seeburger.de',ca_country='DE',ca_province='Baden',ca_locality='Bretten'):
        self.name = name

        #self.pod_name = kubernetesTool.getPod(name)
        self.pod_name = getPod(name)
        if self.pod_name is None:
            self.addPeer(hostname,ca_country,ca_province,ca_locality)
            self.pod_name = getPod(name)
        
        self.listen_port = kubernetesTool.getPort(peername=name, name = prefix + '-listen')
        self.gossip_port = kubernetesTool.getPort(peername=name, name = prefix + '-gossip')
        self.hostname = self.get_hostname()
        self.wallet_tool = None
        
    def get_hostname(self):
        return executePeerCmd(self.pod_name,'echo $HOSTNAME')[:-1]
            
    def addPeer(self, hostname='kubernetes.research.dev.seeburger.de',ca_country='DE',ca_province='Baden',ca_locality='Bretten', prefix='peer'):  
        peername = self.name
        #if peername in peers.keys():
        #    return peers[peername]
        # Create configuration for new peer
        executeLocalCmd(['/bin/bash','-c','cd /home/jovyan/scray-ledger/containers/ && /home/jovyan/scray-ledger/containers/configure-deployment.sh -n ' + peername])
        # Start service
        executeKubectlCmd(['apply', '-f', '/home/jovyan/scray-ledger/containers/target/' + peername + '/k8s-peer-service.yaml'])
        self.listen_port = kubernetesTool.getPort(peername=peername, name = prefix + '-listen')
        self.gossip_port = kubernetesTool.getPort(peername=peername, name = prefix + '-gossip')
        # Create peer configuration
        res = self.createConfig(hostname=hostname,
                                ca_country=ca_country,ca_province=ca_province,ca_locality=ca_locality)
        # Start new peer
        executeKubectlCmd(['apply', '-f','/home/jovyan/scray-ledger/containers/target/' + peername + '/k8s-peer.yaml'])
        #peer = kubernetespeer(name=peername)
        #peers[peername] = self
        #return peer        
        
    def addPeerFromKubernetes():
        peer= kubernetespeer(self.name)
        peers[self.name] = peer
        
    def getCertificate(self):
        try:
            return str(subprocess.check_output(['cat', self.crt]))[2:-3]   
        except Exception as e:
            return str(e)    
        
    def createConfig(self,hostname='kubernetes.research.dev.seeburger.de' ,ca_country='DE',ca_province='Baden',ca_locality='Bretten'):
        try:  
            peername=self.name
            peer_listen_port=self.listen_port
            #peer_gossip_port=self.gossip_port
            peer_gossip_port=self.listen_port
            address = 'peer0.' + peername  + '.' +  hostname + ':'
            output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'create','configmap','hl-fabric-peer-' + peername,
                                              '--from-literal=hostname=' + peername  + '.' + hostname,
                                              '--from-literal=org_name=' + peername,
                                              '--from-literal=data_share=hl-fabric-data-share-service:80',
                                              '--from-literal=ca_country='  + ca_country,
                                              '--from-literal=ca_province=' + ca_province,
                                              '--from-literal=ca_locality=' + ca_locality,
                                              '--from-literal=CORE_PEER_ADDRESS=' + address + peer_listen_port,
                                              '--from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=' + address + peer_gossip_port,
                                              '--from-literal=CORE_PEER_LOCALMSPID=' + peername + 'MSP'])
            return str(output)
        except Exception as e:
            return str(e)  

    
    def deleteConfig(self):
        try:  
            output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'delete','configmap','hl-fabric-peer-' + self.name])
            return str(output)
        except Exception as e:
            return str(e)     
        
    def deleteDeployment(self):
        try:  
            output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'delete','deployment', self.name])
            return str(output)
        except Exception as e:
            return str(e)           

    def deletePeer(self):
        self.deleteDeployment()
        self.deleteConfig()        
        
    def joinHyperledger(self,orderer_ip=None,orderer_hostname=None,orderer_port=None,channel_name=None,shared_fs_host=None,ext_peer_ip='10.15.136.41'):          
        cmd = toCmd(['/mnt/conf/peer_join.sh', orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host,ext_peer_ip])
        print(cmd)
        return executePeerCmd(self.pod_name, cmd)

    def endorse_peer(self,orderer=None,orderer_ip=None,channel_name=None,shared_fs_host=None,ext_peer_ip='10.15.136.41'):
        peername = self.name
        peer_host_name = self.get_hostname()
        #peer_host_name = peername + '.kubernetes.research.dev.seeburger.de'
        print(orderer_ip,channel_name,self.name,shared_fs_host,ext_peer_ip, peer_host_name)
        cmd = toCmd(['/mnt/conf/orderer/scripts/inform_existing_nodes.sh',orderer_ip,channel_name,self.name,shared_fs_host,ext_peer_ip, peer_host_name]) 
        #print(cmd)
        return executePeerCmd(orderer.pod_name,cmd,cli='scray-orderer-cli')

    def endorse_and_join(self,orderer=None,orderer_ip=None,channel_name=None,shared_fs_host=None,ext_peer_ip='10.15.136.41'):    
        res1 = self.endorse_peer(orderer=orderer,orderer_ip=orderer_ip,channel_name=channel_name,shared_fs_host=shared_fs_host)   
        res2  = self.joinHyperledger(orderer_ip=orderer_ip,orderer_hostname=orderer.host,orderer_port=orderer.listen_port,channel_name=channel_name,shared_fs_host=shared_fs_host,ext_peer_ip=ext_peer_ip)
        return res1,res2        
        
    def installAndAproveChaincode(self, orderer=None, channel_name=None, chain_code=None, sequence=1):
        strlist = ['/mnt/conf/install_and_approve_cc.sh', chain_code.host, orderer.get_ip(), orderer.host, str(orderer.listen_port), channel_name, chain_code.pkgid, chain_code.hostname, chain_code.label, chain_code.shared_fs, str(sequence) ]
        return executePeerCmd(self.pod_name,toCmd(strlist)) 

    def commitChaincode(self,channel_name,sequence=1,pkgid=''):
        strlist = ['/mnt/conf/peer/cc_commit.sh', channel_name, str(sequence),pkgid ]
        print(toCmd(strlist))
        return executePeerCmd(self.pod_name,toCmd(strlist))                 
        
       
    def getExport_MSPCONFIGPATH(self,hostname,user='User1'):
        splits = user.split('@')
        hostname =  self.hostname 
        if len(splits) > 1:
            user = splits[0]
            hostname = splits[1]
        path = '/mnt/conf/organizations/peerOrganizations/' + hostname + '/users/' + user + '@' + hostname + '/msp/'
        return 'export CORE_PEER_MSPCONFIGPATH=' + path
        
    def invoke(self,callopt,channel_name=None, user='User1'):
        print(callopt)
        global _out
        _out = widgets.Output()
        with _out:
            try:
                export_cmd = self.getExport_MSPCONFIGPATH(getHostname(peer,user),user=user)
                cmd0 = str('\'' + callopt + "\'")
                #strlist = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem','-C', channel_name, '-n', 'basic', '-c', cmd0]
                #strlist = ['peer', 'chaincode', 'invoke', '--waitForEventTimeout','1s' ,'-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem','-C', channel_name, '-n', 'basic', '-c', cmd0]
                strlist = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--waitForEvent',  '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem','-C', channel_name, '-n', 'basic', '-c', cmd0]

                cmd = toCmd(strlist)
                cmd = export_cmd + '  && ' + cmd
                #print(cmd)  
                output = executePeerCmd(self.pod_name, cmd)
                if output.endswith ('\n'):
                    return output[:-1],_out
                return output,_out
            except Exception as e:
                print('Exception:', e)
                if output.endswith ('\n'):
                    return output[:-1],_out
                return output,_out

    def query(self,callopt, channel_name=None, user='User1'):
        global _out
        _out = widgets.Output()
        with _out:
            try:
                export_cmd = self.getExport_MSPCONFIGPATH(getHostname(peer,user),user=user)
                cmd0 = str('\'' + callopt + "\'")
                strlist = ['peer', 'chaincode', 'query', '-C', channel_name, '-n', 'basic', '-c', cmd0]
                cmd = toCmd(strlist)
                cmd = export_cmd + '  && ' + cmd
                print(cmd)
                output = executePeerCmd(self.pod_name, cmd)
                if output.endswith ('\n'):
                    output=output[:-1]
                return json.loads(output),_out
            except Exception as e:
                print('Exception:', e)
                return output,_out        
        
    def installWalletTool(self,sharepoint=None):
        self.wallet_tool = WalletTool(peer=self,sharepoint=sharepoint)
        self.wallet_tool.installWalletTool()        
        
#class ordererpeer(kubernetes_peer):
class ordererpeer(object):
    def __init__(self, name='',prefix='orderer'):
        #super().__init__(name,prefix)
        self.name = ''
        self.pod_name = kubernetesTool.getPod(name)
        self.host = 'orderer.example.com'
        self.orderer_intern = 7050
        self.listen_port = kubernetesTool.getPort(peername=name, name = prefix + '-listen')
        
    # can change
    def get_ip(self, app='orderer-org1-scray-org'):
        return executeKubectlCmd(['get', 'pods','-l', 'app=' + app,'-o','jsonpath=\'{.items[*].status.podIP}\''])   
    

### Blockchain

In [8]:
class Blockchain(object):
    def __init__(self, name='orderer-org1-scray-org',prefix='orderer', channel_name='invoicing17',shared_fs=None,sharepoint=None):
        self.peers = {}
        
        self.orderer = ordererpeer(name=name,prefix='orderer')
        self.orderer_ip=self.orderer.get_ip()
        self.orderer_hostname=self.orderer.host
        self.orderer_port= self.orderer.listen_port

        self.shared_fs = shared_fs
        self.channel_name=channel_name
        self.shared_fs_host='10.14.128.38:30080'
        self.sharepoint = sharepoint
        self.wallet_tool = WalletTool(sharepoint=self.sharepoint)
 
    def addPeer(self,peername,hostname='kubernetes.research.dev.seeburger.de',ca_country='DE',ca_province='Baden',ca_locality='Bretten'):  
        if peername in self.peers.keys():
            return self.peers[peername]

        peer = KubernetesPeer(name=peername,hostname=hostname,ca_country=ca_country,ca_province=ca_province,ca_locality=ca_locality)
        self.peers[peername] = peer   

    def add_channel(self,name):
        self.channel_name=name
        cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', self.orderer.pod_name, '-c', 'scray-orderer-cli','--', '/bin/sh']
        cmd2 = ['/mnt/conf/orderer/scripts/create_channel.sh', name, 'orderer.example.com', '30081']
        #cmd = toCmd(cmd1 + cmd2)
        ret = subprocess.run(cmd1 + cmd2, stdout = subprocess.PIPE, stderr=subprocess.PIPE)
        #return executePeerCmd(orderer.pod_name, cmd, cli='scray-orderer-cli')
        return ret.stdout.decode('ascii'), ret.stderr.decode('ascii')    
    
    def endorse_and_join_all_peers(self): 
        out2 = widgets.Output()
        with out2:
            for key, peer in self.peers.items():
                peer.endorse_and_join(orderer=self.orderer,orderer_ip=self.orderer_ip,channel_name=self.channel_name,shared_fs_host=self.shared_fs)
        return out2      
    
    
    def deletePeers(self):
        for key,peer in self.peers.items():
            peer.deletePeer()
        for key in list(self.peers.keys()):    
            del self.peers[key]

    # create, delete numbered peers
    def createPeers(self,base_name,start=0, end=10):
        for x in range(start,end):
            peername = base_name + '-' + str(x) 
            self.addPeer(peername)

    def deleteNumberedPeers(name):
        for x in range(0,10):
            key = name + str(x)
            self.peers[key].deletePeer()    
            del self.peers[key]
           
    # kubernetes    
    def deleteAllPeers(config):
        for item in config['items']:
            try:
                if 'org_name' in item['data'].keys():
                    self.deletePeer(item['data']['org_name'])
            except Exception as e:
                print(item['data'])    
         
    def installAndAproveChaincode(self, channel_name=None, chain_code=None):       
        for key, peer in _blockchain.peers.items():
            peer.installAndAproveChaincode(orderer=self.orderer, channel_name=channel_name, chain_code=chain_code)   
            
    def installWalletTool(self):
        for key,peer in self.peers.items():
            peer.installWalletTool(sharepoint=self.sharepoint)
            #wallet_tool = WalletTool(peer=peer,sharepoint=self.sharepoint)
            #wallet_tool.installWalletTool()

In [4]:
import json

class Chaincode(object):
    def __init__(self):
        #self.shared_fs='kubernetes.research.dev.seeburger.de:30080'
        self.shared_fs='10.15.130.111:80'
        self.hostname='invoice-net.org1.example.com'
        self.host='10.14.128.38'
        self.service_name='hl-fabric-cc-external-invoice-net'
        self.port = executeKubectlCmd(['get', 'service', self.service_name, '-o', 'jsonpath="{.spec.ports[?(@.name==\'chaincode\')].nodePort}"'], decode='json')
        self.label='basic_1.0'
        self.pkgid = self.get_pkgid()
        
    def get_pkgid(self):
        try:
            strlist = ['curl','-s','--user','scray:scray','http://' + self.shared_fs + '/cc_descriptions/' + self.hostname + '_' + self.label + '/description-hash.json']
            output = executeLocalCmd(strlist)
            return json.loads(output.decode('ascii'))['description-hash']
        except Exception as e:
            return None   

In [13]:
_blockchain = Blockchain(channel_name='invoicing20',shared_fs='10.15.130.111:80',sharepoint=SharePoint())
cc = Chaincode()

In [14]:
_blockchain.addPeer('aubonmoulin', hostname='fr',ca_country='FR',ca_province='Provence-Alpes-Cote d\'Azur',ca_locality='Malaucene')
_blockchain.addPeer('lyon-impots', hostname='gouv.fr',ca_country='FR',ca_province='Auvergne-Rhone-Alpes',ca_locality='Lyon')
_blockchain.addPeer('austriafactor', hostname='at',ca_country='AT',ca_province='Steiermark',ca_locality='Graz')
_blockchain.addPeer('megustaolive', hostname='es',ca_country='ES',ca_province='Andalusia',ca_locality='Dos Hermanas')

In [ ]:
_blockchain.peers

In [15]:
#cc.pkgid
#_blockchain.wallet_tool.__dict__
_blockchain.installWalletTool()

In [18]:
_blockchain.peers['aubonmoulin'].wallet_tool.addUser('Tony.Dubois','aubonmoulin.fr','aubonmoulin.fr')
_blockchain.peers['aubonmoulin'].wallet_tool.addUser('Stephanie.Hoarau','hotelsaintdenis.re','aubonmoulin.fr')
_blockchain.peers['aubonmoulin'].wallet_tool.addUser('Alexandre.Payet','majolieboutique.net','aubonmoulin.fr')
_blockchain.peers['austriafactor'].wallet_tool.addUser('Emma.Steiner','austriafactor.at','austriafactor.at')
_blockchain.peers['megustaolive'].wallet_tool.addUser('Pedro.Sanchez','megustaolive.es','megustaolive.es')
_blockchain.peers['lyon-impots'].wallet_tool.addUser('Auguste.Boucher','lyon-impots.gouv.fr','lyon-impots.gouv.fr')

('OK: 138 MiB in 84 packages\nOK: 138 MiB in 84 packages\ncreate_csr\nCreate csr  for Auguste.Boucher@lyon-impots.gouv.fr\nConfiguration\n  CA cert path:   ../ca.org1.example.com-cert.pem \n  CA key path:    ../priv_sk  \n  CN of new cert: Auguste.Boucher@lyon-impots.gouv.fr \n  Organisation:    \nOK: 138 MiB in 84 packages\nOK: 138 MiB in 84 packages\npush_csr\nkubernetes.research.dev.seeburger.de:30080\nReplace  http://kubernetes.research.dev.seeburger.de:30080/csrs_to_sign/Auguste.Boucher@lyon-impots.gouv.fr/user.csr  if exits\nConfiguration\n  CA cert path:   ../ca.org1.example.com-cert.pem \n  CA key path:    ../priv_sk  \n  CN of new cert: Auguste.Boucher@lyon-impots.gouv.fr \n  Organisation:    \nOK: 138 MiB in 84 packages\nOK: 138 MiB in 84 packages\npull_csr\nConfiguration\n  CA cert path:   ../ca.org1.example.com-cert.pem \n  CA key path:    ../priv_sk  \n  CN of new cert: Auguste.Boucher@lyon-impots.gouv.fr \n  Organisation:    \nOK: 138 MiB in 84 packages\nOK: 138 MiB in 84

In [ ]:
_blockchain.peers.items()

## cleanup

In [ ]:
_blockchain.deletePeers()
#_blockchain.peers

## init

In [ ]:
strlist = ['curl','-s','--user','scray:scray','http://' + shared_fs + '/cc_descriptions/' + cc_hostname + '_' + cc_label + '/description-hash.json']
cmd = toCmd(strlist)
cmd
output = executeLocalCmd(strlist)
output.decode('ascii')['description-hash']

In [ ]:
_blockchain.peers['aubonmoulin'].__dict__
_blockchain.orderer.__dict__

In [ ]:
executePeerCmd(_blockchain.peers['aubonmoulin'].pod_name,'echo $HOSTNAME')[:-1]

In [ ]:
peers = {}
peers

## add channel

In [ ]:
_blockchain.add_channel('invoicing22')

In [ ]:
_blockchain.__dict__

In [ ]:
out2.outputs[1]['text'].splitlines()

## create peers

### add peers

In [ ]:
_blockchain.peers['aubonmoulin'].__dict__

### endorse and join

In [ ]:
## endorse and join
import ipywidgets as widgets

#orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host

In [ ]:
_blockchain.endorse_and_join_all_peers()

## upload chaincode to sharepoint

In [ ]:
strlist = ['/bin/sh /opt/create-archive.sh',cc.hostname,cc.port,cc.label,cc.shared_fs]
cmd = toCmd(strlist)
print(cmd)
#cmd = export_cmd + '  && ' + cmd
output = executePeerCmd(getPod('cc-deployer'), cmd,cli='cc-deployer')

In [ ]:
output

In [ ]:
cc.__dict__

## install chaincode

In [ ]:
_blockchain.installAndAproveChaincode(channel_name=_blockchain.channel_name, chain_code=cc)

In [ ]:
_blockchain.peers['aubonmoulin'].commitChaincode(_blockchain.channel_name,pkgid=cc.pkgid)

In [ ]:
executePeerCmd(_blockchain.peers['aubonmoulin'].pod_name,'/mnt/conf/peer/cc_commit.sh invoicing20 1 basic_1.0:af21a821cd9c3a5b2784de6cbb278da8c308a72bfd16648f6d333ff51cb2c1f5')

In [ ]:
_blockchain.peers['aubonmoulin'].__dict__

#### obsolete

In [ ]:
for key, peer in _blockchain.peers.items():
    executePeerCmd(peer.pod_name,'apk add curl')
    strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/install_and_approve_cc.sh', '-o', '/mnt/conf/install_and_approve_cc.sh']
    cmd = toCmd(strlist)
    print(cmd)
    output = executePeerCmd(peer.pod_name,  cmd)
    executePeerCmd(peer.pod_name,'chmod 755 /mnt/conf/install_and_approve_cc.sh')
    strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/peer/cc_commit.sh', '-o', '/mnt/conf/peer/cc_commit.sh']
    cmd = toCmd(strlist)
    print(cmd)
    output = executePeerCmd(peer.pod_name,  cmd)
    executePeerCmd(peer.pod_name,'chmod 755 /mnt/conf/peer/cc_commit.sh')

## add roles

In [ ]:
def appendRoles(peers):
    peer=peers['lyon-impots']
    appendRole(peer,user='Auguste.Boucher@lyon-impots.gouv.fr',role='TaxInspector')
    peer_taxinspector=peer
    #taxinspector=get_person_names_with_role('TaxInspector',roles)[0].split(',')[0].split('=')[1]  
    taxinspector='Auguste.Boucher@lyon-impots.gouv.fr'

    identity_taxinspector = getSubmittingClientIdentity(peer_taxinspector, user=taxinspector)[0]

    peer=peers['aubonmoulin']
    appendRole(peer,user='Tony.Dubois@aubonmoulin.fr',role='Seller',taxInspector=identity_taxinspector)
    #appendRole(peer,user='Stephanie.Hoarau',role='Buyer')
    #appendRole(peer,user='Alexandre.Payet',role='Buyer')

    peer=peers['austriafactor']
    appendRole(peer,user='Emma.Steiner@austriafactor.at',role='Factor',taxInspector=identity_taxinspector)

    peer=peers['megustaolive']
    appendRole(peer,user='Pedro.Sanchez@megustaolive.es',role='Buyer',taxInspector=identity_taxinspector)
    appendRole(peer,user='Pedro.Sanchez@megustaolive.es',role='Seller',taxInspector=identity_taxinspector)

In [ ]:
orgs=getPeerOrganizations(_blockchain.peers['aubonmoulin'])

for org in orgs:
    users = getPeerUsers(_blockchain.peers['aubonmoulin'],org)
    print(org,users)


In [ ]:
orgs=getPeerOrganizations(_blockchain.peers['aubonmoulin'])


In [ ]:
orgs

In [ ]:
strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/']
peer=_blockchain.peers['aubonmoulin']
executePeerCmd(peer.pod_name,toCmd(strlist)).splitlines() 

In [ ]:
appendRoles(_blockchain.peers)
#peer=_blockchain.peers['lyon-impots']
#appendRole(peer,user='Auguste.Boucher@lyon-impots.gouv.fr',role='TaxInspector')

In [ ]:
peer=_blockchain.peers['lyon-impots']
getRoles(peer,user='Auguste.Boucher@lyon-impots.gouv.fr')[0]

In [ ]:
callopt='{"function":"AppendRole","Args":["x509::CN=Auguste.Boucher@lyon-impots.gouv.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.lyon-impots.gouv.fr,O=lyon-impots.gouv.fr,L=Lyon,ST=Auvergne-Rhone-Alpes,C=FR","TaxInspector",""]}'

In [ ]:
peer=_blockchain.peers['lyon-impots']
getAllKeys(peer)

In [ ]:
peer=_blockchain.peers['lyon-impots']
callopt= '{"function":"GetAllKeys","Args":[' + ']}'
#peer.query(callopt,channel_name='invoicing18',user='Auguste.Boucher@lyon-impots.gouv.fr')
peer.query(callopt,channel_name='invoicing18',user='User1')

In [ ]:
getAllRoles(peer)
getRoles(peer, user='Auguste.Boucher@lyon-impots.gouv.fr')

In [ ]:
getRoleTransactions(peer)

In [ ]:
invoices = get_invoices()
createInvoices(invoices,_blockchain.peers)
#invoices

In [ ]:
peer=_blockchain.peers['aubonmoulin']
getPeerOrganizations(peer)

In [ ]:
get_peer_of_user('Tony.Dubois@aubonmoulin.fr',_blockchain.peers)

In [ ]:
peer=_blockchain.peers['aubonmoulin']
listInvoices(peer, user='Tony.Dubois@aubonmoulin.fr')[0]

In [ ]:
peer=_blockchain.peers['aubonmoulin']
InvoiceSmartContract(peer=peer, user='Tony.Dubois@aubonmoulin.fr').listInvoices()[0]

In [ ]:
InvoiceSmartContract(peer=peer, user='Tony.Dubois@aubonmoulin.fr').__dict__

In [ ]:
deleteAllInvoices(peer)  